In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %pip install setfit
# %pip install datasets
# %pip install transformers==4.40.2

In [1]:
import pandas as pd
#from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
#from labels import LABELS



# data = "/content/drive/MyDrive/Colab Notebooks/dataset_resume.csv"
# data = "./dataset_resume.csv"
# df = pd.read_csv(data)

# df_new = df.copy()
# # on stocke la classification
# features = df_new[LABELS]
# # création d'une colonne labels pour représenter tous les labels de la description
# df_new["labels"] = pd.Series().fillna("")

: 

In [ ]:
from datasets import Dataset, DatasetDict

def setup(df) -> Dataset:
    for index, row in df.iterrows():
        # s'il y a un résumé, on l'utilise comme description
        df.loc[index, "description"] = (
                row.resume
                if isinstance(row.resume, str)
                else row.description
            )

        features = row[LABELS]
        df.at[index, "labels"] = [feature for feature in features]

    df.drop(columns=['resume', 'title'], inplace=True)
    df.drop(columns=LABELS, inplace=True)
    # suppression de l'index
    df = df.iloc[:, 1:]


    # 90% train, 10% test + validation
    train_testvalid = Dataset.from_pandas(df).train_test_split(test_size=0.2, shuffle=True, seed=42)
    # Split the 10% test + valid in half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
    # gather everyone if you want to have a single DatasetDict
    return Dataset.from_dict({
          'train': train_testvalid['train'],
          'test': test_valid['test'],
          'valid': test_valid['train']
        })

ds = setup(df_new)


In [ ]:
ds = ds.rename_column('description', 'text')
ds = ds.rename_column('labels', 'label')
ds.select([5, 79, 56])

In [ ]:
model = SetFitModel.from_pretrained("dangvantuan/sentence-camembert-base",
                                    multi_target_strategy="multi-output",
                                    use_differentiable_head=True,
                                    head_params={"out_features": 24})
model.model_head

In [ ]:
from transformers import EarlyStoppingCallback

args = TrainingArguments(
    batch_size=3,
    num_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds['train'],
    eval_dataset=ds['valid'],
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5, load_best_model_at_end=True)]
)


In [ ]:
trainer.train()